#Imports

In [0]:
import pandas as pd
from unidecode import unidecode
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

#Lendo os dados

In [0]:
x_train = pd.read_parquet('data/x_train_prepared.parquet')
y_train = pd.read_excel('data/Seguro Saúde - Modelagem.xlsx', sheet_name='MODELAGEM')

# Função que deixa colunas minúsculas
def normalize_columns(df):
    df.columns = [unidecode(col).lower() for col in df.columns]
    return df

# Função para normalizar os valores das colunas de texto
def normalize_text_values(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(lambda x: unidecode(x).lower() if isinstance(x, str) else x)
    return df

y_train = normalize_columns(y_train) 
y_train = normalize_text_values(y_train)

y_train = y_train[['matricula', 'valor']]

display(x_train.head())
display(y_train.head())

#Boruta

In [0]:
rf = RandomForestRegressor(n_jobs=-1, random_state=42)

boruta = BorutaPy(
    estimator=rf,
    n_estimators='auto',
    verbose=2,
    random_state=42,
    max_iter=50
)

boruta.fit(x_train.drop('matricula', axis=1).values, y_train.drop('matricula', axis=1).values.ravel())

In [0]:
selected_features = x_train.drop('matricula', axis=1).columns[boruta.support_].tolist()
print("Selecionadas:", selected_features)


In [0]:
feature_ranking = pd.DataFrame({
    'feature': x_train.drop('matricula', axis=1).columns,
    'ranking': boruta.ranking_,
    'selected': boruta.support_
}).sort_values(by='ranking')

display(feature_ranking)


In [0]:
feature_ranking.to_csv('artifacts/boruta_feature_ranking.csv', index=False)

Concluímos que as 3 variáveis que sao estatiscamente melhores que variáveis de barulho sao:
- Idade
- Fumante
- IMC

#Feature Importance com Random Forest

Queremos confirmar o resultado do boruta vendo o grau de importancia dado pelo random forest

In [0]:
X = x_train.drop('matricula', axis=1)

rf = RandomForestRegressor(random_state=42)
rf.fit(X, y_train.drop('matricula', axis=1))

# Importância das features
importances = rf.feature_importances_
feature_importance = pd.Series(importances, index=x_train.drop('matricula', axis=1).columns).sort_values(ascending=True)

top_features = feature_importance.tail(10)

# Plot
plt.figure(figsize=(10, len(top_features) * 0.35))
top_features.plot(kind='barh')
plt.title("Importância das Features (Random Forest) - TOP 10")
plt.xlabel("Importância", fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=9)
plt.tight_layout()
plt.show()

plt.savefig('plots/rf_feature_importance_top15.png')

Os resultados confirmam o que o Boruta indicava

#Análise de Multicolinearidade

In [0]:
X_selected = x_train[['numeric__idade', 'numeric__imc', 'binary_cat__fumante_1.0']]

plt.figure(figsize=(6, 5))
sns.heatmap(X_selected.corr().abs(), annot=True, cmap='coolwarm', vmin=0, vmax=1)
plt.title("Correlação entre Features Selecionadas")
plt.tight_layout()
plt.show()

plt.savefig('plots/corr_heatmap_selected_features.png')

Vemos aqui que as variáveis selecionadas nao aparesentam correlacao entre si e podem ser mantidas